### Installing depencencies


In [ ]:
ENV["PYTHON"] = "C:\\Users\\lap2r\\AppData\\Local\\Programs\\Python\\Python311\\python.exe"
ENV["PYTHON"] = "C:\\Users\\lap2r\\AppData\\Local\\Programs\\Python\\Python311\\python.exe"

# using Pkg
# Pkg.add("PyCall")
# Pkg.build("PyCall")


# Pkg.add("OrderedCollections")
# Pkg.add("PrettyTables")
# Pkg.add("DataFrames")
# Pkg.add("DotEnv")
# Pkg.add("CSV")
# Pkg.add("Plots")
# Pkg.add("Dates")
# Pkg.add("Gadfly")
# Pkg.add("StatsPlots")
# Pkg.add("StatsBase")
# Pkg.add("JSON")
# Pkg.add("Makie")


# using Conda
# Conda.add("nltk")
# Conda.add("num2words")

### Environment configuration


In [ ]:
using PyCall
pushfirst!(PyVector(pyimport("sys")["path"]), joinpath(@__DIR__, ".."))
paths_rel = pyimport("paths_rel")

alarms_data_file_path = "../" * paths_rel.REL_ALARMS_DATA_FILE
regions_data_file_path = "../" * paths_rel.REL_REGIONS_DATA_FILE
weather_data_file_path = "../" * paths_rel.REL_WEATHER_DATA_FILE
tfidf_csv_path = "../" * paths_rel.REL_ISW_TF_IDF_RESULT

## Alarms dataset analysis


##### Viewing dataset parameters


In [ ]:
using DataFrames
using CSV
using StatsPlots

alarms = DataFrame(CSV.File(alarms_data_file_path))

In [ ]:
size(alarms)

In [ ]:
eltype.(eachcol(alarms))

In [ ]:
describe(alarms)

##### Analysing dataset contents


In [ ]:
# first event date
println("First event date: ", minimum(alarms[!, :start]))

# max event date
println("Latest event date: ", maximum(alarms[!, :end]))

In [ ]:
using Dates
using Statistics

transform!(alarms, [:start, :end] => ((x, y) -> DateTime.(y, "yyyy-mm-dd HH:MM:SS") - DateTime.(x, "yyyy-mm-dd HH:MM:SS")) => :duration)

transform!(alarms, :duration => (x -> Minute.(round.(Int, Dates.value.(x) / (1000 * 60)))) => :duration)

println("minimum duration: ", minimum(alarms[!, :duration]))
println("maximum duration: ", maximum(alarms[!, :duration]))

sort!(alarms, :duration, rev=false)
println("Sorted by minimum duration:")
println(first(alarms, 10))
println("Sorted by maximum duration:")
sort!(alarms, :duration, rev=true)
println(first(alarms, 10))

In [ ]:
using StatsBase;
duration_counts_v1 = countmap(alarms.duration)
# convert Dict{Minute, Int64} to Dict{Int64, Int64}
duration_counts_v1 = Dict{Int64, Int64}(k.value => v for (k, v) in duration_counts_v1)

In [ ]:
# PLOTS_DEFAULTS = Dict(:dpi => 600)

# https://docs.juliaplots.org/latest/generated/attributes_axis/
# https://docs.juliaplots.org/latest/generated/attributes_plot/
# https://docs.juliaplots.org/latest/generated/attributes_subplot/

using Plots.PlotMeasures

default(
    legend=true,
    color=:orangered,
    background_color=:white,
    linecolor=:orangered,
    left_margin=5mm,
    right_margin=5mm,
    top_margin=5mm,
    bottom_margin=5mm,
    xrotation=90,
    draw_arrow=true,
    grid=false,
    minorgrid=false,
    dpi=600,
    size=(800, 800),
)

In [ ]:
using Plots;

max_x = maximum(keys(duration_counts_v1))
max_y = maximum(values(duration_counts_v1))

bar(
    duration_counts_v1,
    title="\nAlarm duration to count",
    xlabel="Duration (minutes)",
    ylabel="Count",
    label="Count",
    xticks=(0:60:max_x+15),
    yticks=(0:15:max_y+15),
    xlims=(-20, max_x + 15),
    ylims=(-10, max_y + 15),
    grid=true,
    minorgrid=true,
)

In [ ]:
duration_counts = countmap(alarms.duration)

# convert dictionary to double array
duration_counts = [k => v for (k, v) in duration_counts]
duration_counts = sort(duration_counts, by=x -> x[2], rev=true)
duration_counts = duration_counts[1:50]

# take each pair and seprate them into 2 arrays first value of pair to first array and second value of pair to second array
duration, counts = [x[1] for x in duration_counts], [x[2] for x in duration_counts]
# create dataframe from items
duration_counts = DataFrame(duration=duration, counts=counts)

In [ ]:
# create a vector of count to region_title
region_counts = countmap(alarms.region_title)
# convert dictionary to double array
region_counts = [k => v for (k, v) in region_counts]
# sort the array by count
region_counts = sort(region_counts, by=x -> x[2], rev=true)
# take each pair and seprate them into 2 arrays first value of pair to first array and second value of pair to second array
region_title, counts = [x[1] for x in region_counts], [x[2] for x in region_counts]
# create dataframe from items
region_counts = DataFrame(region_title=region_title, counts=counts)

In [ ]:
bar(
    region_counts.region_title,
    region_counts.counts,
    xlabel="Region name",
    ylabel="Count",
    title="Alarms Count by region",
    label="Count",
    xrotation=90,
    xticks=:all,
    yticks=0:100:3500,
)

In [ ]:
alarms

In [ ]:
using Dates

transform!(alarms, :start => (x -> Dates.year.(DateTime.(x, "yyyy-mm-dd HH:MM:SS"))) => :year)
transform!(alarms, :start => (x -> Dates.month.(DateTime.(x, "yyyy-mm-dd HH:MM:SS"))) => :month)
transform!(alarms, :start => (x -> Dates.day.(DateTime.(x, "yyyy-mm-dd HH:MM:SS"))) => :day)
transform!(alarms, :start => (x -> Dates.week.(DateTime.(x, "yyyy-mm-dd HH:MM:SS"))) => :week)

alarms[alarms.day .== 1 .&& alarms.month .== 1 .&& alarms.year .== 2023, :week] .= 0
alarms[alarms.day .== 1 .&& alarms.month .== 1 .&& alarms.year .== 2023, :]

In [ ]:
alarms[alarms.year .== 2023, :]

In [ ]:
week_counts_2022 = countmap(alarms[alarms.year .== 2022, :week])
week_2022, counts_2022 = [x[1] for x in week_counts_2022], [x[2] for x in week_counts_2022]

max_x = maximum(week_2022)
max_y = maximum(counts_2022)

bar(
    week_2022,
    counts_2022,
    xlabel="Week",
    ylabel="Count",
    title="Alarms count by week in 2022",
    label="Count",
    xrotation=0,
    xticks=0:1:max_x,
    yticks=0:100:max_y,
    bar_width=0.3,
    size=(1400, 800),
)

In [ ]:
week_counts_2023 = countmap(alarms[alarms.year .== 2023, :week])
week_2023, counts_2023 = [x[1] for x in week_counts_2023], [x[2] for x in week_counts_2023]

max_x = maximum(week_2023)
max_y = maximum(counts_2023)

bar(
    week_2023,
    counts_2023,
    xlabel="Week",
    ylabel="Count",
    title="Alarms count by week in 2023",
    label="Count",
    xrotation=0,
    xticks=0:1:max_x,
    yticks=0:25:max_y,
    bar_width=0.3,
    size=(600, 600),
)

In [ ]:
using Dates
using DataFrames

# copy alarms
df = alarms

# assuming your dataframe is called df
df.duration = map(x -> Dates.Minute(x), df.duration) # convert duration to Minute
# filter for the year 2022
df_2022 = filter(row -> row.year == 2022, df)
# group by week and sum the duration
cumulative_duration = combine(groupby(df_2022, :week), :duration => sum)
# visualize cumulative_duration
bar(
    cumulative_duration[!, :week],
    cumulative_duration[!, :duration_sum],
    xlabel="Week",
    ylabel="Count",
    title="Alarms cumulative duration by week in 2022",
    label="Count",
    bar_width=0.3,
    xticks=0:1:52,
    yticks=0:5000:100000,
    size=(1400, 800),
)

In [ ]:
using Dates
using DataFrames

# copy alarms
df = alarms

# assuming your dataframe is called df
df.duration = map(x -> Dates.Minute(x), df.duration) # convert duration to Minute
# filter for the year 2022
df_2023 = filter(row -> row.year == 2023, df)
# group by week and sum the duration
cumulative_duration = combine(groupby(df_2023, :week), :duration => sum)
# visualize cumulative_duration
bar(
    cumulative_duration[!, :week],
    cumulative_duration[!, :duration_sum],
    xlabel="Week",
    ylabel="Count",
    title="Alarms cumulative duration by week in 2023",
    label="Count",
    bar_width=0.3,
    xticks=0:1:52,
    yticks=0:5000:100000,
    size=(800, 800),
)

In [ ]:
grouped_alarms = combine(groupby(alarms, :region_title), :duration => sum => :duration_sum)
duration_regions = sort(grouped_alarms, :duration_sum, rev=true)

In [ ]:
# visualize duration_regions
bar(
    duration_regions.region_title,
    duration_regions.duration_sum,
    xlabel="Region name",
    ylabel="Duration (minutes)",
    title="Alarms duration by region",
    label="Duration",
    xrotation=90,
    xticks=:all,
    # yticks=0:100:3500,
)

**TF-IDF**


In [ ]:
tfidf = DataFrame(CSV.File(tfidf_csv_path))

println(size(tfidf))
println(names(tfidf))
println(describe(tfidf))

using JSON
tfidf_keywords = tfidf[!, :Keywords]
tfidf_keywords = [JSON.parse(replace(tfidf_keywords[i], "'" => "\""), dicttype=Dict{String,Float64}) for i in 1:length(tfidf_keywords)]

tfidf[!, :Count] = [length(tfidf_keywords[i]) for i in 1:length(tfidf_keywords)]
sort!(tfidf, [:Count], rev=true)


start_date = Date.("2022-02-01", "yyyy-mm-dd")
end_date = Date.(Dates.now())

p = plot(
        tfidf[!, :Date],
        tfidf[!, :Count],
        seriestype=:scatter,
        xlabel="Date",
        ylabel="Count",
        xlims=Dates.value.([start_date, end_date]),
        title="Date vs Unique words count",
        legend=true,
        label="count",
        grid=false,
        size=(1200, 800),
);

xticks!(p, Dates.value.([start_date:Dates.Month(1):end_date;]), Dates.format.([start_date:Dates.Month(1):end_date;], "yyyy-mm"))

p

### ISW Data Analysis


In [ ]:
wordFrequencyToDateFrame = DataFrame()
# word_collection = Dict{String,Int64}()

file_contents = ""
for (root, dirs, files) in walkdir("../0_data_scrapping/results/isw/")
    for file in files
        if endswith(file, ".txt")
            file_path = joinpath(root, file)
            file_content = read(file_path, String)
            file_contents = file_contents * " " * file_content

            words_count = length(split(file_content))
            date = replace(file, "assessment-" => "")
            date = replace(date, ".txt" => "")
            date = Date.(date, "yyyy-mm-dd")
            wordFrequencyToDateFrame = vcat(wordFrequencyToDateFrame, DataFrame(Date=date, Count=words_count))
        end
    end
end

# call do_processing() function in 2_data_prepartion/text_preprocessing.py using pycall

# println(PyCall.python)

# println(joinpath(@__DIR__, "..", "2_data_preparation"))



using PyCall
pushfirst!(PyVector(pyimport("sys")["path"]), joinpath(@__DIR__, "..", "1_data_preparation"))
text_processing = pyimport("text_preprocessing")
file_contents = text_processing.do_preprocessing(file_contents)



wordFrequency = countmap(file_contents)
wordFrequencyToDateFrame = DataFrame(wordFrequency)
# set name for dataframe
rename!(wordFrequencyToDateFrame)
# wordFrequencyToDateFrame
describe(wordFrequencyToDateFrame)



using OrderedCollections

od = OrderedDict(wordFrequency)
od_vector = sort!(od; byvalue=true, rev=true)


od_vector = first(od_vector, 100)
od_vector = map(x -> [x.first, string(x.second)], od_vector)

words_for_table = map(x -> x[1], od_vector)
counts_for_table = map(x -> x[2], od_vector)

Base.displaysize(x::DataFrame) = (100, 100)
dadasdasd = DataFrame(Word=words_for_table, Count=counts_for_table)
dadasdasd.Count = map(x -> parse(Int64, x), dadasdasd.Count)
show(dadasdasd, allrows=true)

In [ ]:
# convert dadasdasd.Count to Int64

# visualize dadasdasd
using Printf

bar(
    dadasdasd.Word,
    dadasdasd.Count,
    xlabel="Word",
    ylabel="Count",
    title="Word frequency",
    label="Number of occurances",
    xrotation=90,
    bar_width=0.3,
    xticks=:all,
    yticks=0:1000:50000,
    yrotation=0,
    yformatter=y -> @sprintf("%d", y),
    aspect_ratio=:none,
    size=(1500, 1000),
)

In [ ]:
# read weather CSV
weather = DataFrame(CSV.File(weather_data_file_path))

In [ ]:
# visualize all attributes city_latitude	city_longitude	city_resolvedAddress	city_address	city_timezone	city_tzoffset	day_datetime	day_datetimeEpoch	day_tempmax	day_tempmin	day_temp	day_feelslikemax	day_feelslikemin	day_feelslike	day_dew	day_humidity	day_precip	day_precipprob	day_precipcover	day_snow	day_snowdepth	day_windgust	day_windspeed	day_winddir	day_pressure	day_cloudcover	day_visibility	day_solarradiation	day_solarenergy	day_uvindex	day_severerisk	day_sunrise	day_sunriseEpoch	day_sunset	day_sunsetEpoch	day_moonphase	day_conditions	day_description	day_icon	day_source	day_preciptype	day_stations	hour_datetime	hour_datetimeEpoch	hour_temp	hour_feelslike	hour_humidity	hour_dew	hour_precip	hour_precipprob	hour_snow	hour_snowdepth	hour_preciptype	hour_windgust	hour_windspeed	hour_winddir	hour_pressure	hour_visibility	hour_cloudcover	hour_solarradiation	hour_solarenergy	hour_uvindex	hour_severerisk	hour_conditions	hour_icon	hour_source	hour_stations
